# Content based
In this case, the idea is to build a recommender that does not use ratings at all, only similarity between books. In order to do that, we need a dataset that contains "keywords" for each book. This is the one used https://zenodo.org/record/4265096#.Ytx7lHZBy3A

In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
df = pd.read_csv('../data/best_books.csv')
df = df[['title', 'genres']]
df

,title,genres
0,The Hunger Games,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas..."
1,Harry Potter and the Order of the Phoenix,"['Fantasy', 'Young Adult', 'Fiction', 'Magic',..."
2,To Kill a Mockingbird,"['Classics', 'Fiction', 'Historical Fiction', ..."
3,Pride and Prejudice,"['Classics', 'Fiction', 'Romance', 'Historical..."
4,Twilight,"['Young Adult', 'Fantasy', 'Romance', 'Vampire..."
...,...,...
52473,Fractured,"['Vampires', 'Paranormal', 'Young Adult', 'Rom..."
52474,Anasazi,"['Mystery', 'Young Adult']"
52475,Marked,"['Fantasy', 'Young Adult', 'Paranormal', 'Ange..."
52476,Wayward Son,"['Fiction', 'Mystery', 'Historical Fiction', '..."


In [3]:
def remove_extra_chars(s):
    s = s.replace("'","")
    s = s.replace("[","")
    s = s.replace("]","")
    return s.replace(",","")

In [4]:
df.genres = df.genres.apply(remove_extra_chars)
df

,title,genres
0,The Hunger Games,Young Adult Fiction Dystopia Fantasy Science F...
1,Harry Potter and the Order of the Phoenix,Fantasy Young Adult Fiction Magic Childrens Ad...
2,To Kill a Mockingbird,Classics Fiction Historical Fiction School Lit...
3,Pride and Prejudice,Classics Fiction Romance Historical Fiction Li...
4,Twilight,Young Adult Fantasy Romance Vampires Fiction P...
...,...,...
52473,Fractured,Vampires Paranormal Young Adult Romance Fantas...
52474,Anasazi,Mystery Young Adult
52475,Marked,Fantasy Young Adult Paranormal Angels Romance ...
52476,Wayward Son,Fiction Mystery Historical Fiction Adventure C...


Given the dataset size and computation power available, the dataset is limited to fantasy books only.

In [5]:
df = df[df.genres.str.contains('Fantasy')]
df.reset_index(drop=True, inplace=True)
df

,title,genres
0,The Hunger Games,Young Adult Fiction Dystopia Fantasy Science F...
1,Harry Potter and the Order of the Phoenix,Fantasy Young Adult Fiction Magic Childrens Ad...
2,Twilight,Young Adult Fantasy Romance Vampires Fiction P...
3,Animal Farm,Classics Fiction Dystopia Fantasy Literature P...
4,The Chronicles of Narnia,Fantasy Classics Fiction Young Adult Childrens...
...,...,...
15316,Nameless Fate,Aliens Science Fiction Romance Erotica BDSM Fa...
15317,Elemental,Fantasy Young Adult Angels Romance Paranormal ...
15318,Fractured,Vampires Paranormal Young Adult Romance Fantas...
15319,Marked,Fantasy Young Adult Paranormal Angels Romance ...


Firstly we need to transform genres to a vector, for which we use CountVectorizor. Then we calculate the cosine similarity between these vectors and save the results to a matrix.

In [6]:
cm = CountVectorizer().fit_transform(df.genres)
cs = cosine_similarity(cm)
cs

array([[1.        , 0.60858062, 0.53311399, ..., 0.38533732, 0.4738791 ,
        0.64605828],
       [0.60858062, 1.        , 0.43259046, ..., 0.40201513, 0.43259046,
        0.54062051],
       [0.53311399, 0.43259046, 1.        , ..., 0.83149718, 0.89473684,
        0.47836487],
       ...,
       [0.38533732, 0.40201513, 0.83149718, ..., 1.        , 0.78258558,
        0.31118796],
       [0.4738791 , 0.43259046, 0.89473684, ..., 0.78258558, 1.        ,
        0.43052839],
       [0.64605828, 0.54062051, 0.47836487, ..., 0.31118796, 0.43052839,
        1.        ]])

Let's test the model on the Lord of the Rings

In [7]:
my_title = 'The Fellowship of the Ring'
#my_title = 'The Lord of the Rings'

In [8]:
df['score'] = cs[df[df.title == my_title].index.to_numpy()[0]]
df.sort_values(by='score', ascending=False).head(20)

C:\Users\chess\AppData\Local\Temp\ipykernel_9820\852289404.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['score'] = cs[df[df.title == my_title].index.to_numpy()[0]]


,title,genres,score
43,The Fellowship of the Ring,Classics Adventure Science Fiction Fantasy Hig...,1.000000
1693,"The Hobbit, Part One",Fantasy Fiction Classics Science Fiction Fanta...,0.932007
5,J.R.R. Tolkien 4-Book Boxed Set: The Hobbit an...,Fantasy Fiction Classics Adventure Science Fic...,0.932007
160,The Lord of the Rings,Fantasy Classics Fiction Adventure Science Fic...,0.932007
5708,Heir of Sea and Fire,Fantasy Fiction Science Fiction Fantasy High F...,0.932007
255,The Return of the King,Fantasy Fiction Classics Adventure Science Fic...,0.907265
280,The Two Towers,Fantasy Fiction Classics Adventure Science Fic...,0.907265
1771,Cursor's Fury,Fantasy Fiction Epic Fantasy High Fantasy Magi...,0.901441
2789,Magician's Gambit,Fantasy Fiction Epic Fantasy Science Fiction F...,0.901441
569,The Sword of Shannara,Fantasy Fiction Epic Fantasy Science Fiction F...,0.901441


Results contain mostly other books from the Lord of the Rings franchise.